In [1]:
!pip install ultralytics kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00


In [2]:
import kagglehub
import os
import shutil
import random
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
raw_path = kagglehub.dataset_download("nirmalsankalana/crop-pest-and-disease-detection")
base_dir = '/content/datasets/crop_disease'
if os.path.exists(base_dir): shutil.rmtree(base_dir)

100%|██████████| 1.25G/1.25G [00:33<00:00, 39.5MB/s]

Extracting files...


In [6]:
all_classes = sorted([d for d in os.listdir(raw_path) if os.path.isdir(os.path.join(raw_path, d))])

print(f"Found {len(all_classes)} Classes: {all_classes}")

Found 22 Classes: ['Cashew anthracnose', 'Cashew gumosis', 'Cashew healthy', 'Cashew leaf miner', 'Cashew red rust', 'Cassava bacterial blight', 'Cassava brown spot', 'Cassava green mite', 'Cassava healthy', 'Cassava mosaic', 'Maize fall armyworm', 'Maize grasshoper', 'Maize healthy', 'Maize leaf beetle', 'Maize leaf blight', 'Maize leaf spot', 'Maize streak virus', 'Tomato healthy', 'Tomato leaf blight', 'Tomato leaf curl', 'Tomato septoria leaf spot', 'Tomato verticulium wilt']


In [7]:
# Split Data (80% Train, 20% Validation)
for class_name in all_classes:
    # Create directories
    os.makedirs(f"{base_dir}/train/{class_name}", exist_ok=True)
    os.makedirs(f"{base_dir}/val/{class_name}", exist_ok=True)

    # Get images
    src_folder = os.path.join(raw_path, class_name)
    images = [f for f in os.listdir(src_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    random.shuffle(images)

    # Split
    split_idx = int(len(images) * 0.8)
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    # Copy files
    for img in train_imgs:
        shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'train', class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(src_folder, img), os.path.join(base_dir, 'val', class_name, img))

In [8]:
model = YOLO('yolo11n-cls.pt')

In [9]:
print("Starting Training...")
results = model.train(
    data=base_dir,
    epochs=10,           # Reduced for speed
    imgsz=224,           # Standard classification size
    batch=16,
    project='crop_project',
    name='crop_model'
)

Starting Training...
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/crop_disease, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=crop_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=T

In [11]:
print("\n" + "="*40)
print("🔑 TARGET VARIABLE MAPPING (Save for PPT)")
print("="*40)
print(results)
print("="*40 + "\n")


🔑 TARGET VARIABLE MAPPING (Save for PPT)
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e2fb7d34440>
curves: []
curves_results: []
fitness: 0.9474101960659027
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8960111141204834, 'metrics/accuracy_top5': 0.998809278011322, 'fitness': 0.9474101960659027}
save_dir: PosixPath('/content/runs/classify/crop_project/crop_model')
speed: {'preprocess': 0.11084928061131052, 'inference': 0.6425845237149161, 'loss': 0.0003283038297013624, 'postprocess': 0.0024965064494283785}
task: 'classify'
top1: 0.8960111141204834
top5: 0.998809278011322



In [13]:
# Load the trained model to get the class names
model = YOLO('/content/runs/classify/crop_project/crop_model/weights/best.pt')

print(model.names)

{0: 'Cashew anthracnose', 1: 'Cashew gumosis', 2: 'Cashew healthy', 3: 'Cashew leaf miner', 4: 'Cashew red rust', 5: 'Cassava bacterial blight', 6: 'Cassava brown spot', 7: 'Cassava green mite', 8: 'Cassava healthy', 9: 'Cassava mosaic', 10: 'Maize fall armyworm', 11: 'Maize grasshoper', 12: 'Maize healthy', 13: 'Maize leaf beetle', 14: 'Maize leaf blight', 15: 'Maize leaf spot', 16: 'Maize streak virus', 17: 'Tomato healthy', 18: 'Tomato leaf blight', 19: 'Tomato leaf curl', 20: 'Tomato septoria leaf spot', 21: 'Tomato verticulium wilt'}


In [14]:
from google.colab import files
import os

model_path = '/content/runs/classify/crop_project/crop_model/weights/best.pt'

if os.path.exists(model_path):
    print(f"Found model at: {model_path}")
    print(" Downloading now...")
    files.download(model_path)
else:
    print(f"Error: File not found at {model_path}")
    print("Check the 'runs' folder in the file browser on the left.")

Found model at: /content/runs/classify/crop_project/crop_model/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>